In [1]:
import os
import sys

from src.utility.project_setup import setup_project_path

setup_project_path()


Changed to project root directory


'/workspaces/Approval_Predict'

Load collected data

In [2]:
from pathlib import Path
import pandas as pd

root = setup_project_path()
file_path = Path(root) / "outputs" / "datasets" / "collection" / "loan_approval.csv"

if not file_path.exists():
    raise FileNotFoundError(f"Dataset not found at: {file_path}")

df = pd.read_csv(file_path).drop(['name', 'city'], axis=1)
df.head(5)

No 'jupyter_notebooks' directory found the the current path


,income,credit_score,loan_amount,years_employed,points,loan_approved
0,113810,389,39698,27,50.0,False
1,44592,729,15446,28,55.0,False
2,33278,584,11189,13,45.0,False
3,127196,344,48823,29,50.0,False
4,66048,496,47174,4,25.0,False


Data Exploration

In [3]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

[]

Generate a profiling report for columns with missing data

In [4]:
from ydata_profiling import ProfileReport
if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

There are no variables with missing data


Correlation and PPS Analysis

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20, 6), font_annot=8):

    print("\n")
    print("* Analyse how the target variable for your ML models are correlated with other variables (features and target)")
    print("* Analyse multi-colinearity, that is, how the features are correlated among themselves")

    # Create a figure with 3 subplots side by side
    fig, axes = plt.subplots(1, 3, figsize=(24, 8))
    
    print("\n")
    print("*** Correlation and PPS Heatmaps ***")
    
    # Spearman Correlation
    print("\nSpearman Correlation - evaluates monotonic relationship")
    mask_spearman = np.zeros_like(df_corr_spearman, dtype=bool)
    mask_spearman[np.triu_indices_from(mask_spearman)] = True
    mask_spearman[abs(df_corr_spearman) < CorrThreshold] = True
    
    sns.heatmap(df_corr_spearman, annot=True, xticklabels=True, yticklabels=True,
                mask=mask_spearman, cmap='viridis', annot_kws={"size": font_annot}, 
                ax=axes[0], linewidth=0.5)
    axes[0].set_yticklabels(df_corr_spearman.columns, rotation=0)
    axes[0].set_title('Spearman Correlation', fontsize=14, fontweight='bold')
    
    # Pearson Correlation
    print("\nPearson Correlation - evaluates linear relationship")
    mask_pearson = np.zeros_like(df_corr_pearson, dtype=bool)
    mask_pearson[np.triu_indices_from(mask_pearson)] = True
    mask_pearson[abs(df_corr_pearson) < CorrThreshold] = True
    
    sns.heatmap(df_corr_pearson, annot=True, xticklabels=True, yticklabels=True,
                mask=mask_pearson, cmap='viridis', annot_kws={"size": font_annot}, 
                ax=axes[1], linewidth=0.5)
    axes[1].set_yticklabels(df_corr_pearson.columns, rotation=0)
    axes[1].set_title('Pearson Correlation', fontsize=14, fontweight='bold')
    
    # PPS
    print("\nPower Predictive Score - detects linear and non-linear relationships (0-1 scale)")
    mask_pps = np.zeros_like(pps_matrix, dtype=bool)
    mask_pps[abs(pps_matrix) < PPS_Threshold] = True
    
    sns.heatmap(pps_matrix, annot=True, xticklabels=True, yticklabels=True,
                mask=mask_pps, cmap='rocket_r', annot_kws={"size": font_annot},
                linewidth=0.05, linecolor='grey', ax=axes[2])
    axes[2].set_title('Predictive Power Score (PPS)', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

In [7]:
DisplayCorrAndPPS(df_corr_pearson = df_corr_pearson,
                  df_corr_spearman = df_corr_spearman, 
                  pps_matrix = pps_matrix,
                  CorrThreshold = 0.4, PPS_Threshold =0.2,
                  figsize=(12,10), font_annot=10)

NameError: name 'df_corr_pearson' is not defined